# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Jun  2 2022 12:31PM  238656        10   AERO601-01   
1  Jun  2 2022 12:31PM  238656        10   AERO601-03   
2  Jun  2 2022 12:31PM  238656        10   AERO601-04   
3  Jun  2 2022 12:24PM  238657        19  ADV80003070   
4  Jun  2 2022 12:24PM  238657        19  ADV80003071   
5  Jun  2 2022 12:02PM  238654        16  SEQ-8095665   
6  Jun  2 2022 11:56AM  238655        19   DEX0006902   
7  Jun  2 2022 11:56AM  238655        19   DEX0006903   
8  Jun  2 2022 11:56AM  238655        19   DEX0006904   
9  Jun  2 2022 11:56AM  238655        19   DEX0006905   

                                    Customer ShipmentStatus  
0                               Aerogen LTD.       Released  
1                               Aerogen LTD.       Released  
2                               Aerogen LTD.       Released  
3                        AdvaGen Pharma, Ltd       Released  
4                        AdvaGen Pharma, Ltd       Released  
5                                    Seqirus       Released  
6  Dexcel Pharma Technologies Ltd. DBA WDSrx      Executing  
7  Dexcel Pharma Technologies Ltd. DBA WDSrx       Released  
8  Dexcel Pharma Technologies Ltd. DBA WDSrx      Executing  
9  Dexcel Pharma Technologies Ltd. DBA WDSrx       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
47  238654       Released          1
48  238655      Executing         12
49  238655       Released          6
50  238656       Released          3
51  238657       Released          2

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238653                NaN        1.0       NaN
238654                NaN        NaN       1.0
238655                NaN       12.0       6.0
238656                NaN        NaN       3.0
238657                NaN        NaN       2.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238544                0.0       19.0       2.0
238553                0.0        1.0       0.0
238556                0.0        0.0       2.0
238557                0.0        0.0       5.0
238564                0.0        6.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238544                  0         19         2
238553                  0          1         0
238556                  0          0         2
238557                  0          0         5
238564                  0          6         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238544          0         19         2
1               238553          0          1         0
2               238556          0          0         2
3               238557          0          0         5
4               238564          0          6         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               238544                  19        2
1               238553                   1         
2               238556                            2
3               238557                            5
4               238564                   6        1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0   Jun  2 2022 12:31PM  238656        10   
3   Jun  2 2022 12:24PM  238657        19   
5   Jun  2 2022 12:02PM  238654        16   
6   Jun  2 2022 11:56AM  238655        19   
24  Jun  2 2022 11:48AM  238653        12   
25  Jun  2 2022 11:46AM  238652        10   
27  Jun  2 2022 11:24AM  238650        19   
28  Jun  2 2022 11:23AM  238649        10   
52  Jun  2 2022 11:12AM  238647        10   
53  Jun  2 2022 10:53AM  238645        19   

                                     Customer  
0                                Aerogen LTD.  
3                         AdvaGen Pharma, Ltd  
5                                     Seqirus  
6   Dexcel Pharma Technologies Ltd. DBA WDSrx  
24                                  Hush Hush  
25                           Methapharm, Inc.  
27                      VITABIOTICS  USA INC.  
28                           Eywa Pharma Inc.  
52                           Eywa Pharma Inc.  
53                        AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Jun  2 2022 12:31PM  238656        10   
1  Jun  2 2022 12:24PM  238657        19   
2  Jun  2 2022 12:02PM  238654        16   
3  Jun  2 2022 11:56AM  238655        19   
4  Jun  2 2022 11:48AM  238653        12   
5  Jun  2 2022 11:46AM  238652        10   
6  Jun  2 2022 11:24AM  238650        19   
7  Jun  2 2022 11:23AM  238649        10   
8  Jun  2 2022 11:12AM  238647        10   
9  Jun  2 2022 10:53AM  238645        19   

                                    Customer Completed Executing Released  
0                               Aerogen LTD.                            3  
1                        AdvaGen Pharma, Ltd                            2  
2                                    Seqirus                            1  
3  Dexcel Pharma Technologies Ltd. DBA WDSrx                  12        6  
4                                  Hush Hush                   1           
5                           Methapharm, Inc.                            2  
6                      VITABIOTICS  USA INC.                            1  
7                           Eywa Pharma Inc.                           24  
8                           Eywa Pharma Inc.                            1  
9                        AdvaGen Pharma, Ltd                           10

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Jun  2 2022 12:31PM  238656        10   
1  Jun  2 2022 12:24PM  238657        19   
2  Jun  2 2022 12:02PM  238654        16   
3  Jun  2 2022 11:56AM  238655        19   
4  Jun  2 2022 11:48AM  238653        12   
5  Jun  2 2022 11:46AM  238652        10   
6  Jun  2 2022 11:24AM  238650        19   
7  Jun  2 2022 11:23AM  238649        10   
8  Jun  2 2022 11:12AM  238647        10   
9  Jun  2 2022 10:53AM  238645        19   

                                    Customer Released Executing Completed  
0                               Aerogen LTD.        3                      
1                        AdvaGen Pharma, Ltd        2                      
2                                    Seqirus        1                      
3  Dexcel Pharma Technologies Ltd. DBA WDSrx        6        12            
4                                  Hush Hush                  1            
5                           Methapharm, Inc.        2                      
6                      VITABIOTICS  USA INC.        1                      
7                           Eywa Pharma Inc.       24                      
8                           Eywa Pharma Inc.        1                      
9                        AdvaGen Pharma, Ltd       10

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  Jun  2 2022 12:31PM  238656        10   
1  Jun  2 2022 12:24PM  238657        19   
2  Jun  2 2022 12:02PM  238654        16   
3  Jun  2 2022 11:56AM  238655        19   
4  Jun  2 2022 11:48AM  238653        12   

                                    Customer Released Executing Completed  
0                               Aerogen LTD.        3                      
1                        AdvaGen Pharma, Ltd        2                      
2                                    Seqirus        1                      
3  Dexcel Pharma Technologies Ltd. DBA WDSrx        6        12            
4                                  Hush Hush                  1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse  \
0  Jun  2 2022 12:31PM  238656        10   
1  Jun  2 2022 12:24PM  238657        19   
2  Jun  2 2022 12:02PM  238654        16   
3  Jun  2 2022 11:56AM  238655        19   
4  Jun  2 2022 11:48AM  238653        12   

                                    Customer  Released  Executing  Completed  
0                               Aerogen LTD.       3.0        NaN        NaN  
1                        AdvaGen Pharma, Ltd       2.0        NaN        NaN  
2                                    Seqirus       1.0        NaN        NaN  
3  Dexcel Pharma Technologies Ltd. DBA WDSrx       6.0       12.0        NaN  
4                                  Hush Hush       NaN        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  Jun  2 2022 12:31PM  238656        10   
1  Jun  2 2022 12:24PM  238657        19   
2  Jun  2 2022 12:02PM  238654        16   
3  Jun  2 2022 11:56AM  238655        19   
4  Jun  2 2022 11:48AM  238653        12   

                                    Customer  Released  Executing  Completed  
0                               Aerogen LTD.       3.0        0.0        0.0  
1                        AdvaGen Pharma, Ltd       2.0        0.0        0.0  
2                                    Seqirus       1.0        0.0        0.0  
3  Dexcel Pharma Technologies Ltd. DBA WDSrx       6.0       12.0        0.0  
4                                  Hush Hush       0.0        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2147673      57.0       18.0        6.0
12         1193013       8.0        4.0        0.0
15         1670310      31.0       35.0       15.0
16          954490       2.0        2.0        0.0
19         1193210      22.0       33.0       31.0
20         1431491       5.0       27.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2147673      57.0       18.0        6.0
1        12  1193013       8.0        4.0        0.0
2        15  1670310      31.0       35.0       15.0
3        16   954490       2.0        2.0        0.0
4        19  1193210      22.0       33.0       31.0
5        20  1431491       5.0       27.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      57.0       18.0        6.0
1        12       8.0        4.0        0.0
2        15      31.0       35.0       15.0
3        16       2.0        2.0        0.0
4        19      22.0       33.0       31.0
5        20       5.0       27.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   57.0
1        12  Released    8.0
2        15  Released   31.0
3        16  Released    2.0
4        19  Released   22.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16    19    20
Status                                     
Completed   6.0  0.0  15.0  0.0  31.0   0.0
Executing  18.0  4.0  35.0  2.0  33.0  27.0
Released   57.0  8.0  31.0  2.0  22.0   5.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16    19    20
0          Completed   6.0  0.0  15.0  0.0  31.0   0.0
1          Executing  18.0  4.0  35.0  2.0  33.0  27.0
2           Released  57.0  8.0  31.0  2.0  22.0   5.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16    19    20
0  Completed   6.0  0.0  15.0  0.0  31.0   0.0
1  Executing  18.0  4.0  35.0  2.0  33.0  27.0
2   Released  57.0  8.0  31.0  2.0  22.0   5.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()